<a href="https://colab.research.google.com/github/nerdk312/Model-based-RL/blob/master/CURL_Pacman_110520_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
'''
function ClickConnect(){
console.log("Working"); 
document.querySelector("colab-toolbar-button#connect").click() 
}
setInterval(ClickConnect,60000)
'''

# Used to save Pacman video

In [0]:
#remove " > /dev/null 2>&1" to see what is going on under the hood
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

In [0]:
!apt-get update > /dev/null 2>&1
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools 2>&1
!pip install ez_setup > /dev/null 2>&1
!pip install gym[atari] > /dev/null 2>&1
!apt-get install ffmpeg

Requirement already up-to-date: setuptools in /usr/local/lib/python3.6/dist-packages (46.1.3)
Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.6-0ubuntu0.18.04.1).
0 upgraded, 0 newly installed, 0 to remove and 32 not upgraded.


# Installation and Imports

In [25]:
#!pip install git+git://github.com/mila-iqia/atari-representation-learning.git
#!pip install git+git://github.com/ankeshanand/pytorch-a2c-ppo-acktr-gail
!pip install git+git://github.com/openai/baselines
!pip install wandb

  Cloning git://github.com/openai/baselines to /tmp/pip-req-build-5ksu29oe
  Running command git clone -q git://github.com/openai/baselines /tmp/pip-req-build-5ksu29oe
  Created wheel for baselines: filename=baselines-0.1.6-cp36-none-any.whl size=220664 sha256=a19503f4726b98183b5903aedae4cd537822f4fe5dd918870df4cd8f5e1be811
  Stored in directory: /tmp/pip-ephem-wheel-cache-powxntfx/wheels/42/1c/91/28314e0cd1d2cc57cf8dd18b20c4c9a0f39ae518adc13caf24
Successfully built baselines


In [26]:
!wandb login ##############

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
Successfully logged in to Weights & Biases!


In [0]:
import os
from __future__ import print_function
import pickle
import sys
sys.path.append('/content/drive/My Drive')
import wandb

import random
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
from tqdm import tqdm
import datetime
import time

from Contrastive_loss_Pacman.custom_wrappers import custom_wrapper
from Contrastive_loss_Pacman.encoder import make_encoder
from Contrastive_loss_Pacman.EarlyStopping import EarlyStopping_loss
from Contrastive_loss_Pacman.GeneralFunctions import General_functions
from Contrastive_loss_Pacman.utils import make_dir, random_crop,center_crop_image, soft_update_params, weight_init
from torch.autograd import Variable

import gym

# Needed to create dataloaders
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

# Utils

In [0]:
class ReplayBuffer(Dataset):
    """Buffer to store environment transitions."""
    def __init__(self, obs_shape,action_shape, state_shape,capacity, batch_size, device,image_size=84,rand_crop = True,transform=None):
        self.capacity = capacity
        self.batch_size = batch_size
        self.device = device
        self.image_size = image_size
        self.rand_crop = rand_crop
        self.transform = transform

        obs_dtype = np.uint8 # Need to consider the sign of this
        self.obses = np.empty((capacity, *obs_shape), dtype=obs_dtype)
        self.actions = np.empty((capacity,*action_shape),dtype = np.float32)
        self.state_changes = np.empty((capacity, *state_shape), dtype = np.float32) 

        self.idx = 0
        self.last_save = 0
        self.full = False

    def add(self, obs,action,state_change): # Nawid- Add information to replay buffer
        
        np.copyto(self.obses[self.idx], obs)
        np.copyto(self.actions[self.idx], action)
        np.copyto(self.state_changes[self.idx], state_change)

        self.idx = (self.idx + 1) % self.capacity # This makes the data gets replaced in a recursive manner when the capacity is full
        self.full = self.full or self.idx == 0
    
    def sample_cpc(self): # Nawid - samples images I believe

        start = time.time()
        idxs = np.random.randint(
            0, self.capacity if self.full else self.idx, size=self.batch_size
        ) # Used to randomly sample indices

        obses = self.obses[idxs] # Nawid - Samples observation
        pos = obses.copy() # Nawid -

        # Random crop or centre crops the image
        if self.rand_crop:
            obses_input = random_crop(obses,self.image_size)
        else:
            obses_input = center_crop_image(obses, self.image_size)  

        # Nawid - Crop images randomly
        obses = random_crop(obses, self.image_size)
        pos = random_crop(pos, self.image_size)

        obses_input = torch.tensor(obses_input, device= self.device).float()
        obses = torch.as_tensor(obses, device=self.device).float()
        pos = torch.as_tensor(pos, device=self.device).float()
        actions = torch.as_tensor(self.actions[idxs], device=self.device)
        state_changes = torch.as_tensor(self.state_changes[idxs], device=self.device) 
        
        cpc_kwargs = dict(obs_anchor=obses, obs_pos=pos,
                          time_anchor=None, time_pos=None) # Nawid  Postitive example is pos whilst anchor is obses

        return obses_input, actions, state_changes, cpc_kwargs
        
    def save(self, save_dir):
        path = os.path.join(save_dir, '%d_%d.pt' % (self.last_save, self.capacity))
        payload = [
            self.obses[0:self.capacity], #  Changed the payload compared to their training as I intend to use a offline training at the moment
            self.actions[0:self.capacity],
            self.state_changes[0:self.capacity]
        ]
        torch.save(payload, path)
    
    def load(self, save_dir): # Nawid - Loads the data into the replay buffer
        chunks = os.listdir(save_dir)
        chucks = sorted(chunks, key=lambda x: int(x.split('_')[0]))
        for chunk in chucks:
            start, end = [int(x) for x in chunk.split('.')[0].split('_')]
            path = os.path.join(save_dir, chunk)
            payload = torch.load(path)
            assert self.idx == start
            self.obses[start:end] = payload[0]
            self.actions[start:end] = payload[1]
            self.state_changes[start:end] = payload[2]
            self.idx = end

    def __getitem__(self, idx): # Nawid - Obtains item from replay buffer
        ''' Remove the randomness in the dataloading of each sample as the dataloader itself should be able to find the different values
        idx = np.random.randint(
            0, self.capacity if self.full else self.idx, size=1
        )
        idx = idx[0]
        '''
        obses = np.expand_dims(self.obses[idx],0) # Need to expand dim to allow it to be the shape for cropping, then need to squeeze so its a 4d tensor rather than 5d with an extra dim so it can be used with the dataloader
        pos = obses.copy()
        #center crop image
        if self.rand_crop:
            obses_input = random_crop(obses,self.image_size) #center_crop_image(obses,self.image_size) # 
        else:
            obses_input = center_crop_image(obses, self.image_size)

        # random crop images
        obses = random_crop(obses, self.image_size)
        pos = random_crop(pos, self.image_size)

        obses = np.squeeze(obses)
        pos = np.squeeze(pos)

        action = self.actions[idx]
        state_change = self.state_changes[idx]

        if self.transform:
            obses = self.transform(obses)

        cpc_kwargs = dict(obs_anchor=obses, obs_pos=pos) # Nawid  Postitive example is pos whilst anchor is obses
                          
        return obses_input, action, state_change, cpc_kwargs
        
    def __len__(self):
        return self.capacity

# CURL

In [0]:
class CURL(nn.Module): # Nawid - Module for the contrastive loss
    """
    CURL
    """
    def __init__(self, obs_shape, z_dim, encoder_feature_dim, num_layers, num_filters):
        super(CURL, self).__init__()

        # Need to fix the encoders since I do not plan to use the critics
        self.encoder = make_encoder( # Nawid - Encoder of critic which is also used for the contrastive loss
            obs_shape, encoder_feature_dim, num_layers,
            num_filters, output_logits=True)

        self.encoder_target = make_encoder(obs_shape, encoder_feature_dim, num_layers, 
                                           num_filters, output_logits=True)
        
        self.encoder_target.load_state_dict(self.encoder.state_dict()) # copies the parameters of the encoder into the target encoder which is changing slowly
        self.W = nn.Parameter(torch.rand(z_dim, z_dim)) # Nawid - weight vector for the bilinear product

    def encode(self, x, detach=False, ema=False):
        """
        Encoder: z_t = e(x_t)
        :param x: x_t, x y coordinates
        :return: z_t, value in r2
        """
        if ema:
            with torch.no_grad():
                z_out = self.encoder_target(x)
        else:
            z_out = self.encoder(x)

        if detach:
            z_out = z_out.detach()
        return z_out
    
    def compute_logits(self, z_a, z_pos): # Nawid -  computes logits for contrastive loss
        """
        Uses logits trick for CURL:
        - compute (B,B) matrix z_a (W z_pos.T)
        - positives are all diagonal elements
        - negatives are all other elements
        - to compute loss use multiclass cross entropy with identity matrix for labels
        """
        Wz = torch.matmul(self.W, z_pos.T)  # (z_dim,B)
        logits = torch.matmul(z_a, Wz)  # (B,B)
        logits = logits - torch.max(logits, 1)[0][:, None]
        return logits


class Dynamics_model(nn.Module):
    ''' MLP network'''
    def __init__(self, obs_shape,hidden_dim,
                 output_dim , encoder_feature_dim, num_layers, num_filters):
        super(Dynamics_model,self).__init__()

        self.encoder = make_encoder(obs_shape, encoder_feature_dim,num_layers,num_filters,output_logits=True)

        self.trunk = nn.Sequential(
            nn.Linear(self.encoder.feature_dim + 4 , hidden_dim),nn.BatchNorm1d(num_features=hidden_dim), nn.ReLU(), # Size of the input is related to the encoder output as well as the concatenated one hot vector for the actions
            nn.Linear(hidden_dim, hidden_dim), nn.BatchNorm1d(num_features=hidden_dim), nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)
        )
        self.apply(weight_init)
    
    def forward(self, obs,aux, detach_encoder = False):
        obs = self.encoder(obs, detach = detach_encoder) 
        obs = torch.cat((obs, aux), 1) # Join vectors along this dimension
        prediction = self.trunk(obs)        
        return prediction


In [0]:
class CurlAgent(object):
    ''' CURL representation learning'''
    def __init__(
        self,
        obs_shape,
        device,
        hidden_dim = 256,
        output_dim = 2,
        encoder_feature_dim = 50,
        encoder_lr = 1e-3,
        encoder_tau = 0.005,
        num_layers=4,
        num_filters = 32,
        cpc_update_freq=1,
        encoder_update_freq = 2,
        detach_encoder=False,
    ):
        self.device = device
        self.cpc_update_freq = cpc_update_freq
        self.image_size = obs_shape[-2] # Changed this to the numpy dimension
        self.detach_encoder = detach_encoder
        self.encoder_update_freq = encoder_update_freq
        self.encoder_tau = encoder_tau
        self.epoch_step = 0
        
        self.CURL = CURL(obs_shape, encoder_feature_dim,
                         encoder_feature_dim, num_layers, num_filters).to(self.device)
        
        self.Model = Dynamics_model(obs_shape,hidden_dim,output_dim,encoder_feature_dim,num_layers,num_filters).to(self.device)

        # tie encoders between and CURL and dynamics predictor - VERY IMPORTANT-  CAN REMOVE IF I WANT SEPARATE NETWORKS
        self.Model.encoder.copy_conv_weights_from(self.CURL.encoder)

        self.cpc_optimizer = torch.optim.Adam(
                self.CURL.parameters(), lr=encoder_lr
            )
        
        self.dynamics_optimizer = torch.optim.Adam(self.Model.parameters(), lr= encoder_lr)

        self.cross_entropy_loss = nn.CrossEntropyLoss()
        self.MSE_loss = nn.MSELoss() # Nawid - Added this loss for the prediction
        self.train()

        #wandb.watch(self.CURL, log='all') #  Seems that I cannot use the wand.watch for the CURL model or the dynamics model(when the weights are shared between the dynamics and curl model)
        #wandb.watch(self.Model, log='all')
    
    def train(self, training = True):
        self.training = training
        self.CURL.train(training)
        self.Model.train(training)

    def update(self, training_replay_buffer,val_dataloader,early_stopper,step):
        #torch.cuda.empty_cache() # Releases cache so the GPU has more memory
        if early_stopper.early_stop:
            print('early stopping')
            return

        obs, actions_one_hot,predicted_change, cpc_kwargs = training_replay_buffer.sample_cpc()
        
        if step % self.encoder_update_freq == 0:
            soft_update_params(
                self.CURL.encoder, self.CURL.encoder_target,
                self.encoder_tau
            )
        
        if step % self.cpc_update_freq == 0:
            
            obs_anchor, obs_pos = cpc_kwargs["obs_anchor"], cpc_kwargs["obs_pos"]
            self.update_cpc(obs_anchor, obs_pos,cpc_kwargs) # Nawid -  Performs the contrastive loss I believe
        
        if step % 2 ==0:    
            self.update_dynamics(obs, actions_one_hot,predicted_change)

        if step % 200 == 0:
            self.validation(val_dataloader,early_stopper)

    def update_cpc(self, obs_anchor, obs_pos, cpc_kwargs):
        z_a = self.CURL.encode(obs_anchor) # Nawid -  Encode the anchor
        z_pos = self.CURL.encode(obs_pos, ema=True) # Nawid- Encode the positive with the momentum encoder

        logits = self.CURL.compute_logits(z_a, z_pos) #  Nawid- Compute the logits between them
        labels = torch.arange(logits.shape[0]).long().to(self.device)
        loss = self.cross_entropy_loss(logits, labels)
        wandb.log({'Contrastive Training loss':loss.item()})

        self.cpc_optimizer.zero_grad()
        loss.backward()

        self.cpc_optimizer.step()  # Nawid - Used to update the cpc
    
    def update_dynamics(self, obs,actions, labels):
        prediction = self.Model(obs,actions,detach_encoder=True) # gradient not backpropagated to the encoder
        prediction_loss = self.MSE_loss(prediction,labels)
        wandb.log({'Dynamics Training loss':prediction_loss.item()}) #  Need to use .item otherwise the loss will still be kept which will reduce the memory on the GPU
        
        self.dynamics_optimizer.zero_grad()
        prediction_loss.backward()
        self.dynamics_optimizer.step()

    def validation(self, dataloader,early_stopper):
        epoch_contrastive_loss = 0
        epoch_dynamics_loss = 0 
        self.Model.eval()
        self.CURL.eval()
        with torch.no_grad():
            for i, (obses, actions, state_change, cpc_kwargs) in enumerate(dataloader):
                obs_anchor, obs_pos = cpc_kwargs["obs_anchor"], cpc_kwargs["obs_pos"]

                obses = obses.float().to(self.device)
                obs_anchor = obs_anchor.float().to(self.device)
                obs_pos = obs_pos.float().to(self.device)
                actions = actions.to(self.device)
                state_change = state_change.to(self.device)
                
                z_a = self.CURL.encode(obs_anchor) # Nawid -  Encode the anchor
                z_pos = self.CURL.encode(obs_pos, ema=True) # Nawid- Encode the positive with the momentum encoder
                logits = self.CURL.compute_logits(z_a, z_pos) #  Nawid- Compute the logits between them
                labels = torch.arange(logits.shape[0]).long().to(self.device)
                loss = self.cross_entropy_loss(logits, labels)
                epoch_contrastive_loss += loss.item()
                
                val_prediction = self.Model(obs_anchor,actions,detach_encoder=False) # gradient not backpropagated to the encoder
                val_prediction_loss = self.MSE_loss(val_prediction,state_change)
                epoch_dynamics_loss += val_prediction_loss.item()
            
            average_epoch_contrastive_loss = epoch_contrastive_loss/i
            average_epoch_dynamics_loss = epoch_dynamics_loss/i            
            
            self.epoch_step += 1 # increase epoch counter
            wandb.log({'Contrastive Validation loss':average_epoch_contrastive_loss,'Dynamics Validation loss':average_epoch_dynamics_loss,'epoch': self.epoch_step})
            print('epoch:', self.epoch_step)
            print('val prediction:', val_prediction[0:20]) #  batch of the val prediction is the size of the last batch so it will be what is leftover till the set is complete
            print('state change:',state_change[0:20])

            early_stopper(average_epoch_dynamics_loss,self.Model,self.dynamics_optimizer)

        self.train()

def make_agent(obs_shape, device, dict_info):
    return CurlAgent(
        obs_shape = obs_shape,
        device = device,
        encoder_update_freq =dict_info['encoder_update_freq'],
        encoder_feature_dim = dict_info['encoder_feature_dim'],
        encoder_lr = dict_info['encoder_lr'],
        encoder_tau = dict_info['encoder_tau'],
        num_layers = dict_info['num_layers'],
        num_filters = dict_info['num_filters']
    )

# Data collection and preprocessing

In [0]:
class Data_collection(General_functions):
    def __init__(self, ENV_NAME, n_actions, possible_positions,info_dict,buffer_capacity):
        super(Data_collection, self).__init__(ENV_NAME, n_actions, possible_positions)
        self.grayscale = info_dict['grayscale']
        self.channels = 1 if self.grayscale else 3 
        self.obs_shape = (info_dict['image_size'], info_dict['image_size'],self.channels*info_dict['frame_stack']) # Nawid - Stack together images (multiply of 3 present due to 3 channels (RGB))
        self.pre_aug_obs_shape = (info_dict['pre_transform_image_size'],info_dict['pre_transform_image_size'],self.channels*info_dict['frame_stack'])
        self.action_shape = (n_actions,)   
        self.state_shape = (2,)     
        self.buffer_capacity = buffer_capacity
        self.replay_buffer  = ReplayBuffer(self.pre_aug_obs_shape,self.action_shape,self.state_shape,self.buffer_capacity, info_dict['batch_size'],self.device, info_dict['image_size'],info_dict['random_crop'])
        self.ts = time.gmtime()
        self.ts = time.strftime('%d-%m_%H:%M')
        self.walls_present = info_dict['walls_present']


    def random_action_selection(self, state,prev_action = None):
        while True:
            action = np.random.randint(1,5) 
            feasible_action = self.next_position(state,action)
            if feasible_action:
                self.prev_action_counter = True
                return action, None
            else:
                infeasible_action_one_hot = self.one_hot(action)
                if self.prev_action_counter and prev_action !=None:
                    return prev_action, infeasible_action_one_hot 
    
    def gather_random_trajectories(self,num_traj):
        work_dir = '/content/drive/My Drive/MsPacman-data' + '/' + self.ts +'_capacity-' + str(self.buffer_capacity) +'_grayscale-'+ str(self.grayscale) + '_walls_present-'+ str(self.walls_present)
        work_dir = make_dir(work_dir)

        for n in range(num_traj):            
            if n % 10 ==0:
                print('trajectory number:',n)
                # Initial set up
            #self.env.seed(0)

            self.env = gym.make(self.ENV_NAME) # Due to error in code, I reinstantiate the env each time
            self.env = custom_wrapper(self.env,grayscale = self.grayscale)
            obs = self.env.reset()
            
            self.repeated_end = False
            info_labels = self.env.labels() # Nawid - Used to get the current state
            state = self.state_conversion(info_labels) # Used to get the initial state
            prev_action = None # Initialise prev action has having no action
            
            while True:
                sampled_action, infeasible_action_one_hot = self.random_action_selection(state,prev_action)
                sampled_action_one_hot = self.one_hot(sampled_action)
                
                next_obs, reward, done, next_info = self.env.step(sampled_action)
                next_info_labels = next_info['labels']
                
                next_state = self.state_conversion(next_info_labels)
                state_change = next_state -  state
                
                self.check_state(state,next_state)
                self.check_all_agents(info_labels, next_info_labels) # need to use the info labels to predict the state as the info labels have all the informaiton
                    
                if not self.repeated_end:            
                    if infeasible_action_one_hot is not None and self.walls_present:
                        fake_next_state = np.zeros_like(state) #  Need to instantiate a new version each time to prevent updating a single variable which will affect all places(eg lists) where the variable is added
                        fake_next_state[0:-2] = next_state[0:-2].copy() # the enemy position of the fake next state is the current enemy position
                        fake_next_state[-2:] = state[-2:].copy() # The agent position for the fake next state is the state before any action was taken
                        fake_state_change = fake_next_state - state
                        self.replay_buffer.add(obs,infeasible_action_one_hot, fake_state_change)

                    self.replay_buffer.add(obs,sampled_action_one_hot, state_change)
                else:
                    done = True   
                
                obs = next_obs # do not need to copy as a new variable of obs is instantiated at each time step.
                state = next_state.copy()
                info_labels = next_info_labels.copy()
                prev_action = sampled_action

                if done:
                    break    
                if self.replay_buffer.full:
                    self.replay_buffer.save(work_dir)
                    return

# Hyperparameters

In [50]:
ENV_NAME = 'MsPacmanDeterministic-v4'
n_actions = 4 #9 - Nawid - Change to 5 actions as the 4 other actions are simply copies of the other actions, therefore 5 actions should lower the amount of data needed.
parse_dict= {'pre_transform_image_size':100,
             'image_size':84,
             'frame_stack':4,
             'train_capacity':1000,
             'val_capacity':1000,
             'num_train_steps':20000,
             'batch_size':256,
             'random_crop': True,
             'encoder_update_freq':2,
             'encoder_feature_dim':50,
             'encoder_lr':1e-3,
             'encoder_tau':0.05,
             'num_layers':4,
             'num_filters':32,
             'load_trajectories':True,
             'grayscale': True,
             'pretrain_model': False,
             'walls_present': False
            }

wandb.init(entity="nerdk312", project="Contrastive_learning",config = parse_dict)

possible_positions = np.load('/content/drive/My Drive/MsPacman-data/possible_pacman_positions.npy',allow_pickle=True)
preloaded_train_data = '/content/drive/My Drive/MsPacman-data/11-05_16:56-grayscale-True-walls_present-False' #'/content/drive/My Drive/MsPacman-data/Data_obstacles_present/80000_10-05_13:35/buffer'
preloaded_val_data =  '/content/drive/My Drive/MsPacman-data/11-05_17:02_capacity-1000_grayscale-True_walls_present-False_capacity-' #'/content/drive/My Drive/MsPacman-data/Data_obstacles_present/10000_09-05_17:05/buffer' 

config = wandb.config
if parse_dict['load_trajectories']:
    config.loaded_train_trajectories = preloaded_train_data
    config.loaded_val_trajectories = preloaded_val_data

if parse_dict['pretrain_model']:
    config.pretrained_model = pretrain_model_dir

# Main -  Model training

Data - Collection

In [51]:
data_object = Data_collection(ENV_NAME,n_actions,possible_positions, parse_dict,parse_dict['train_capacity'])
val_data_object = Data_collection(ENV_NAME,n_actions,possible_positions, parse_dict, parse_dict['val_capacity'])

if parse_dict['load_trajectories']:
    data_object.replay_buffer.load(preloaded_train_data)
    val_data_object.replay_buffer.load(preloaded_val_data)
else:
    data_object.gather_random_trajectories(5000)
    val_data_object.gather_random_trajectories(5000)

val_dataloader = DataLoader(val_data_object.replay_buffer, batch_size = 256, shuffle = True)

cpu
cpu


In [0]:
agent = make_agent(
        obs_shape = data_object.obs_shape,
        device =data_object.device,
        dict_info = parse_dict
    )

dyn_model_name = 'Dynamics'+ '_' + data_object.ts
early_stopping_dynamics = EarlyStopping_loss(patience=3, verbose=True, wandb=wandb, name=dyn_model_name)

env = gym.make(ENV_NAME)
env = custom_wrapper(env, grayscale=True)
obs = env.reset()
info_labels = env.labels()
state = data_object.state_conversion(info_labels)

for step in range(parse_dict['num_train_steps']):
    if early_stopping_dynamics.early_stop: #  Stops the training if early stopping counter is hit
        break

    if step >= 0:
        agent.update(data_object.replay_buffer,val_dataloader,early_stopping_dynamics,step)